In [39]:
import os
import cv2
import json
import numpy as np

def load_dataset(data_dir):
    # Initialize lists to store images and annotations
    images = []
    annotations = []
    bbox_list = []
    keypoints_list = []

    # Load annotations from COCO-style JSON file
    with open(os.path.join(data_dir, '_annotations.coco.json'), 'r') as f:
        data = json.load(f)

    # Create a dictionary to map image IDs to their annotations
    image_id_to_annotations = {}
    for annotation in data['annotations']:
        image_id = annotation['image_id']
        if image_id not in image_id_to_annotations:
            image_id_to_annotations[image_id] = []
        image_id_to_annotations[image_id].append(annotation)

    # Load images and corresponding annotations
    for img_info in data['images']:
        img_path = os.path.join(data_dir, img_info['file_name'])
        img = cv2.imread(img_path)
        images.append(img)

        img_annotations = image_id_to_annotations.get(img_info['id'], [])
        for ann in img_annotations:
            bbox_list.append(ann["bbox"])
            keypoints_list.append(ann["keypoints"])
    return images, bbox_list, keypoints_list

import numpy as np
from sklearn.model_selection import train_test_split

# Load dataset
data_dir = 'lateral face cephalometry.v1i.coco/train'
images, bbox_list, keypoints_list = load_dataset(data_dir)
print(len(images), len(bbox_list), len(keypoints_list))
# Split dataset into training and validation sets
X_train, X_val, bbox_train, bbox_val, keypoints_train, keypoints_val = train_test_split(images, bbox_list, keypoints_list, test_size=0.2, random_state=42)

# Convert lists to NumPy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
bbox_train = np.array(bbox_train)
bbox_val = np.array(bbox_val)
keypoints_train = np.array(keypoints_train)
keypoints_val = np.array(keypoints_val)


print("Number of training samples:", len(X_train))
print("Number of validation samples:", len(X_val))


39 36 36


ValueError: Found input variables with inconsistent numbers of samples: [39, 36, 36]

In [ ]:
def load_dataset(data_dir):
    # Initialize lists to store images and annotations
    images = []
    annotations = []
    bbox_list = []
    keypoints_list = []

    # Load annotations from COCO-style JSON file
    with open(os.path.join(data_dir, '_annotations.coco.json'), 'r') as f:
        data = json.load(f)

    # Create a dictionary to map image IDs to their annotations
    image_id_to_annotations = {}
    for annotation in data['annotations']:
        image_id = annotation['image_id']
        if image_id not in image_id_to_annotations:
            image_id_to_annotations[image_id] = []
        image_id_to_annotations[image_id].append(annotation)

    # Load images and corresponding annotations
    for img_info in data['images']:
        img_path = os.path.join(data_dir, img_info['file_name'])
        img = cv2.imread(img_path)
        images.append(img)

        img_annotations = image_id_to_annotations.get(img_info['id'], [])
        bbox_list.append(img_annotations["bbox"])
        keypoints_list.append(img_annotations["keypoints"])

    return images, bbox_list, keypoints_list


In [31]:
# Load and preprocess your dataset
# (Assuming you have already loaded your dataset into X_train, y_train, X_val, y_val)
# X_train and X_val should contain your images, and y_train and y_val should contain your annotations
import tensorflow as tf
from tensorflow.keras import layers, models

def create_custom_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(4, activation='linear'))  # Assuming output shape is [nose_x, nose_y, mouth_x, mouth_y]
    return model

model = create_custom_model(X_train.shape[1:])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train your model
history = model.fit(X_train, bbox_train, keypoints_train, validation_data=(X_val, bbox_val,keypoints_val), batch_size=32, epochs=10)

# Monitor training
# Optionally, you can plot the training and validation loss curves
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


NameError: name 'bbox_train' is not defined